## dand_p5_fraudDetection

In [1]:
import sys
sys.version

'2.7.13 |Anaconda custom (x86_64)| (default, Dec 20 2016, 23:05:08) \n[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]'

## The original code 

## Extract the 1st part of that code

* Read file into pandas and then use dataframe to do all kinds of things like in kaggle

In [1]:
#!/usr/bin/python

import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
features_list = ['poi','salary'] # You will need to use more features
features_list = [] # create a new list to incorporate the columns

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)
#     print len()
#     for key, value in data_dict.items():
#         print key , ':\t' , value
#         print '\n\n'
    dataset_dict = pd.read_pickle('final_project_dataset.pkl')
        
# read them into pandas dataframe
dataset_df_temp = pd.DataFrame(dataset_dict)
dataset_df = pd.DataFrame.transpose(dataset_df_temp)
dataset_df.shape
dataset_df

# get the features and have a look
print features_list
print len(features_list)

/Users/Tree/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[]
0


## 1. Clean dataset
* most of the are numerical data, so can use some ways to do this 
* email and poi are not, do it later
* use KNN from fancyimputation package to fillin the NaN (fancyimpute is not included generally, so fill with 0)

In [2]:
dataset_df['poi'].value_counts()

False    128
True      18
Name: poi, dtype: int64

### 1.1. Drop outlier

In [3]:
print 'Before drop shape \t\t: ' , dataset_df.shape
print 'Before drop the row is \t\t: ' , dataset_df.query(''' salary > 2.5e7 and salary != 'NaN' ''').index
dataset_df.drop(dataset_df.query(''' salary > 2.5e7 and salary != 'NaN' ''').index, inplace=True)
print 'After drop shape \t\t : ' , dataset_df.shape
print 'After drop the row is \t\t: ' , dataset_df.query(''' salary > 2.5e7 and salary != 'NaN' ''').index

Before drop shape 		:  (146, 21)
Before drop the row is 		:  Index([u'TOTAL'], dtype='object')
After drop shape 		 :  (145, 21)
After drop the row is 		:  Index([], dtype='object')


In [4]:
dataset_df = dataset_df.convert_objects(convert_numeric=True)

In [82]:
print dataset_df.dtypes

bonus                        float64
deferral_payments            float64
deferred_income              float64
director_fees                float64
email_address                 object
exercised_stock_options      float64
expenses                     float64
from_messages                float64
from_poi_to_this_person      float64
from_this_person_to_poi      float64
loan_advances                float64
long_term_incentive          float64
other                        float64
poi                             bool
restricted_stock             float64
restricted_stock_deferred    float64
salary                       float64
shared_receipt_with_poi      float64
to_messages                  float64
total_payments               float64
total_stock_value            float64
dtype: object


In [83]:
# we can see this is also an outlier
dataset_df.query(''' total_payments > 1e8 ''').shape
dataset_df.drop(dataset_df.query(''' total_payments > 1e8 ''').index, inplace=True)

In [84]:
dataset_df.shape

(144, 21)

## 1.4. try do copy the True values 
> The labels are very skewed and the algorithm cannot get good combination of precision and recall
in order to get enought precison and recall, I append some of the true values 

> Turns out this worked. 
Used 15 times, gets me 0.412 and 0.404 as precision and recall respectively

In [87]:
poi_trues = dataset_df.query(''' poi == True ''')
print poi_trues.shape

(17, 21)


In [88]:
dataset_df_temp = dataset_df
for ii in range(16):
    dataset_df_temp = dataset_df_temp.append(poi_trues)
dataset_df_dup = dataset_df_temp
print dataset_df_dup.shape


(416, 21)


## ====================  Now the dataset is much better     ====================

### 1.2. the NaN in this table are simply the string saying 'NaN'
* <font color='red'>Remember don't change the NaN in </font>

In [89]:
# try to use the dataset with replica of True values
dataset_df_removeNaN_zero = dataset_df_dup.replace('NaN', 0, inplace=False)

In [90]:
# fill 'NaN' with 0
# dataset_df_removeNaN_zero = dataset_df.replace('NaN', 0, inplace=False)

# fill 'NaN' with np.nan
# dataset_df_removeNaN_npnan = dataset_df.replace('NaN', np.nan, inplace=False)

In [91]:
# have a look at the table with 0 replacing the NaN
dataset_df_removeNaN_zero.head(2)

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# not good at all
# because there are too many 'NaN', so if use 0 to fill
# there will be too much 0 and this would have a bad influence on the performance
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
ALLEN PHILLIP K,4175000.0,2869717.0,-3081055.0,0.0,phillip.allen@enron.com,1729541.0,13868.0,2195.0,47.0,65.0,...,304805.0,152.0,False,126027.0,-126027.0,201955.0,1407.0,2902.0,4484442.0,1729541.0
BADUM JAMES P,0.0,178980.0,0.0,0.0,0,257817.0,3486.0,0.0,0.0,0.0,...,0.0,0.0,False,0.0,0.0,0.0,0.0,0.0,182466.0,257817.0


In [92]:
# have a look at the table with np.nan replacing the NaN
# dataset_df_removeNaN_npnan.head(5)

In [93]:
# collect the numerical columns to get ready for imputation
# leave the string type columns alone 

num_var = []
non_num_var = []
for each_column in dataset_df_removeNaN_zero:    
    if each_column != 'email_address' and each_column != 'poi':
        num_var.append(each_column)
    else :
        non_num_var.append(each_column)

print 'num_var\t\t' , num_var
print 'non_num_var\t' , non_num_var

num_var		['bonus', 'deferral_payments', 'deferred_income', 'director_fees', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances', 'long_term_incentive', 'other', 'restricted_stock', 'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi', 'to_messages', 'total_payments', 'total_stock_value']
non_num_var	['email_address', 'poi']


In [95]:
dataset_df_removeNaN_zero['poi'].value_counts()

True     289
False    127
Name: poi, dtype: int64

#### Get the numerical values for scaling

In [97]:
dataset_df_removeNaN_zero_num = dataset_df_removeNaN_zero[num_var]

### 1.3. Scaling the numerical data

In [98]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

ss = StandardScaler()
dataset_filled_knn_df_scaled_vals = ss.fit_transform(dataset_df_removeNaN_zero_num.values)
dataset_filled_knn_df_scaled = pd.DataFrame(dataset_filled_knn_df_scaled_vals, \
                                            index=dataset_df_removeNaN_zero_num.index,\
                                            columns=dataset_df_removeNaN_zero_num.columns)


In [99]:
dataset_filled_knn_df_scaled.head(2)

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
ALLEN PHILLIP K,1.968379,4.485466,-2.548274,-0.182251,-0.313287,-0.874188,1.853617,-0.179612,0.170923,-0.057789,-0.318114,-0.431808,-0.642469,-0.198181,-0.313615,0.237190,0.660394,1.079383,-0.425047
BADUM JAMES P,-0.817250,0.017295,0.485190,-0.182251,-0.504224,-1.128066,-0.277831,-0.780736,-0.352967,-0.057789,-0.757238,-0.432087,-0.716930,-0.033258,-1.229791,-0.866993,-0.763059,-0.811414,-0.598031


### 2. Feature engineering

#### 2-way interaction numerical features

In [100]:
print 'number of columns before interaction' , len(dataset_filled_knn_df_scaled.columns)

original_num_of_cols = len(dataset_filled_knn_df_scaled.columns)

# 2 way plus, minus, times
for outer_iter in range(original_num_of_cols):
    for inner_iter in range(outer_iter + 1, original_num_of_cols):
        first_name = dataset_filled_knn_df_scaled.columns[outer_iter]
        last_name = dataset_filled_knn_df_scaled.columns[inner_iter]
        dataset_filled_knn_df_scaled[first_name + '_' + last_name + '_' + 'plus'] \
                        = dataset_filled_knn_df_scaled[first_name] + dataset_filled_knn_df_scaled[last_name]
            
        dataset_filled_knn_df_scaled[first_name + '_' + last_name + '_' + 'minus'] \
                        = dataset_filled_knn_df_scaled[first_name] - dataset_filled_knn_df_scaled[last_name]
            
        dataset_filled_knn_df_scaled[first_name + '_' + last_name + '_' + 'times'] \
                        = dataset_filled_knn_df_scaled[first_name] * dataset_filled_knn_df_scaled[last_name]
        
        dataset_filled_knn_df_scaled[first_name + '_' + last_name + '_' + 'divide'] \
                        = dataset_filled_knn_df_scaled[first_name] / dataset_filled_knn_df_scaled[last_name]
            

            
            
            
dataset_filled_knn_df_scaled.shape

print 'number of columns after interaction' , len(dataset_filled_knn_df_scaled.columns)

number of columns before interaction 19
number of columns after interaction 703


In [101]:
dataset_filled_knn_df_scaled.shape

(416, 703)

#### Scale again

In [102]:
ss = StandardScaler()
dataset_filled_knn_df_scaled_vals_again_vals = ss.fit_transform(dataset_filled_knn_df_scaled.values)
dataset_filled_knn_df_scaled_vals_again = pd.DataFrame(dataset_filled_knn_df_scaled_vals_again_vals, \
                                            index=dataset_filled_knn_df_scaled.index,\
                                            columns=dataset_filled_knn_df_scaled.columns)

### 3. Modeling 

#### StratifiedShuffleSplit due to the skewedness

In [76]:
X_all = dataset_filled_knn_df_scaled_vals_again
Y_all = dataset_df_dup['poi']

#### Tried so many times, found dtc is the most potential one, so put work on decision tree classifier
#### Could further tune it, but use as default can get the result as required, so I guess it okay for now

### 3.1 Choose a model from all the models
* Importing
* Creating
* reshape to np array for the StratifiedShuffleSplit

In [77]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

mlp = MLPClassifier()
knc = KNeighborsClassifier()
svc = SVC()
gpc = GaussianProcessClassifier()
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
abc = AdaBoostClassifier()
nbc = GaussianNB()

X_array = np.array(X_all.values)
Y_array = np.array(Y_all.reshape(-1,1))
sss = StratifiedShuffleSplit(n_splits=100, test_size=0.1, random_state=2017)
for train_index, test_index in sss.split(X_array, Y_array):
    x_train, x_test = X_array[train_index], X_array[test_index]
    y_train, y_test = Y_array[train_index], Y_array[test_index]

    mlp.fit(x_train, y_train)
    knc.fit(x_train, y_train)
    svc.fit(x_train, y_train)
    gpc.fit(x_train, y_train)
    dtc.fit(x_train, y_train)
    rfc.fit(x_train, y_train)
    abc.fit(x_train, y_train)
    nbc.fit(x_train, y_train)
    
    
#     from sklearn.metrics import classification_report
#     clf = dtc
#     y_pred = clf.predict(x_test).ravel()
#     y_true = y_test
#     print classification_report(y_true, y_pred)

    

In [78]:
print (classification_report(y_true, y_pred)).split()[-4:-1]

['1.00', '1.00', '1.00']


In [79]:
pre_rec_f1 = classification_report(y_true, y_pred).split()[-4:-1]
int_pre_rec_f1 = [float(ii) for ii in pre_rec_f1]
print int_pre_rec_f1

[1.0, 1.0, 1.0]


### 3.2. Grid Search after choose decision tree model

In [80]:
from sklearn.model_selection import GridSearchCV

params = {'max_depth': [5, 8, 10, 15, 20],
          'min_samples_split': [2, 4, 6, 8],
          'min_samples_leaf': [1, 2, 3, 4],
          'random_state': [2017]}
gsearch = GridSearchCV(estimator = dtc, 
                       param_grid = params, scoring='roc_auc', n_jobs=-1, iid=False, cv=5)

gsearch.fit(X_all, Y_all.ravel())

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=False, n_jobs=-1,
       param_grid={'min_samples_split': [2, 4, 6, 8], 'random_state': [2017], 'max_depth': [5, 8, 10, 15, 20], 'min_samples_leaf': [1, 2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [81]:
print gsearch.best_params_
print gsearch.best_estimator_
print gsearch.best_score_

{'min_samples_split': 2, 'random_state': 2017, 'max_depth': 10, 'min_samples_leaf': 2}
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=2,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=2017, splitter='best')
0.954904183536


In [35]:
from sklearn.metrics import classification_report

y_pred = gsearch.predict(x_test).ravel()
y_true = y_test
print classification_report(y_true, y_pred)

             precision    recall  f1-score   support

      False       1.00      1.00      1.00        13
       True       1.00      1.00      1.00        29

avg / total       1.00      1.00      1.00        42



### Following are the format thing
* Convert all of the results to the form required and output for the review

#### For the output format, I have to append the poi to the first column

In [194]:
dataset_filled_knn_df_scaled.insert(loc=0, column='poi', value=dataset_df_dup['poi'].values)

In [230]:
features_list = list(dataset_filled_knn_df_scaled.columns)

In [231]:
my_dataset = dataset_filled_knn_df_scaled.transpose().to_dict()

In [233]:
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)
dump_classifier_and_data(clf, my_dataset, features_list)

[[ 0.          3.194632    3.52404595 ...,  1.3790878  -0.21008601
  -6.90812155]
 [ 0.         -0.56496234 -0.05547462 ..., -0.07514396  0.26669429
   1.15647901]
 [ 0.         -0.56496234 -0.29357392 ..., -0.81099109 -0.14182925
  -0.45908994]
 ..., 
 [ 0.         -0.56496234 -0.29357392 ..., -0.12513246  0.31808184
   1.24784545]
 [ 1.         -0.56496234 -0.29357392 ..., -2.41850053 -0.93427216
  -0.24929258]
 [ 0.         -0.56496234 -0.29357392 ..., -0.11373478  0.29993124
   1.230355  ]]


In [318]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
x = np.array([[1,2,3], [4,5,6], [7,8,9], [2,4,6], [7,4,2], [2,3,4], [1,1,1], [2,2,2], [4,6,3], [9,4,2]])
y = np.array([[1], [0], [1], [1], [0], [1], [0], [1], [0], [1]])
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=2017)

print x.shape
print y.shape
for train_index, test_index in sss.split(x, y):
    x_tr, x_te = x[train_index], x[test_index]
    y_tr, y_te = y[train_index], y[test_index]
    
    print x_tr.shape, x_te.shape

# It shuffles but not stratified, means it could be a subset has no True lable at all
# That's not good for the model to fit because the label set is skewed

(10, 3)
(10, 1)
(8, 3) (2, 3)
(8, 3) (2, 3)
(8, 3) (2, 3)
(8, 3) (2, 3)
(8, 3) (2, 3)
